In [14]:
import numpy as np
import ee
import io
from google.api_core import exceptions, retry
import google.auth
from models import get_model

PROJECT = "pc530-fao-fra-rss"  # change to your cloud project name
# ee.Initialize(project=PROJECT)

## INIT WITH HIGH VOLUME ENDPOINT
credentials, _ = google.auth.default()
ee.Initialize(
credentials,
project=PROJECT,
opt_url="https://earthengine-highvolume.googleapis.com",
)

In [15]:
def get_ee_img(coords):
    """retrieve s2 image composite from ee at given coordinates. coords is a tuple of (lon, lat) in degrees."""
    ## MAKE S2 COMPOSITE IN HEXAGONS ##########################################
    # Using Cloud Score + for cloud/cloud-shadow masking
    # Harmonized Sentinel-2 Level 2A collection.
    s2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

    # Cloud Score+ image collection. Note Cloud Score+ is produced from Sentinel-2
    # Level 1C data and can be applied to either L1C or L2A collections.
    csPlus = ee.ImageCollection("GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED")

    # Use 'cs' or 'cs_cdf', depending on your use case; see docs for guidance.
    QA_BAND = "cs_cdf"

    # The threshold for masking; values between 0.50 and 0.65 generally work well.
    # Higher values will remove thin clouds, haze & cirrus shadows.
    CLEAR_THRESHOLD = 0.50

    # Make a clear median composite.
    sampleImage = (
        s2.filterDate("2023-01-01", "2023-12-31")
        .filterBounds(ee.Geometry.Point(coords[0], coords[1]).buffer(64*10)) # only images touching 64 pixel centroid buffer
        .linkCollection(csPlus, [QA_BAND])
        .map(lambda img: img.updateMask(img.select(QA_BAND).gte(CLEAR_THRESHOLD)))
        .median()
        .select(["B4", "B3", "B2", "B8"], ["R", "G", "B", "N"])
    )
    return sampleImage

@retry.Retry()
def get_patch(coords, image, format="NPY"):
    """Uses ee.data.ComputePixels() to get a 32x32 patch centered on the coordinates, as a numpy array."""
    
    # Output resolution in meters.
    SCALE = 10

    # Pre-compute a geographic coordinate system.
    proj = ee.Projection("EPSG:4326").atScale(SCALE).getInfo()

    # Get scales in degrees out of the transform.
    SCALE_X = proj["transform"][0]
    SCALE_Y = -proj["transform"][4]

    # Patch size in pixels.
    PATCH_SIZE = 32

    # Offset to the upper left corner.
    OFFSET_X = -SCALE_X * PATCH_SIZE / 2
    OFFSET_Y = -SCALE_Y * PATCH_SIZE / 2
    
    REQUEST = {
        "fileFormat": "NPY",
        "grid": {
            "dimensions": {"width": PATCH_SIZE, "height": PATCH_SIZE},
            "affineTransform": {
                "scaleX": SCALE_X,
                "shearX": 0,
                "shearY": 0,
                "scaleY": SCALE_Y,
            },
            "crsCode": proj["crs"],
        },
    }
    
    request = dict(REQUEST)
    request["fileFormat"] = format
    request["expression"] = image
    request["grid"]["affineTransform"]["translateX"] = coords[0] + OFFSET_X
    request["grid"]["affineTransform"]["translateY"] = coords[1] + OFFSET_Y
    return np.load(io.BytesIO(ee.data.computePixels(request)))

In [44]:
id, latlon = 1233804841, [102.19,-1.54]#[-257.82, -1.54]#[-172.3490007781034,-13.523357265222518] #[-60.25204,3.86655]#
print(latlon)
image = get_ee_img(latlon)
patch = get_patch(latlon, image)
# print(patch)

[102.19, -1.54]


In [45]:
import geemap
Map = geemap.Map()
Map.addLayer(image, {"bands": ["R", "G", "B"], "min": 0, "max": 2000}, "S2")
Map.addLayer(ee.Geometry.Point(latlon), {"color": "red"}, "Centroid")
Map.centerObject(ee.Geometry.Point(latlon), 18)
Map

Map(center=[-1.54, 102.19000000000001], controls=(WidgetControl(options=['position', 'transparent_bg'], widget…

In [46]:
import tensorflow as tf
from numpy.lib.recfunctions import structured_to_unstructured
unstruct = structured_to_unstructured(patch)
print(unstruct)

[[[ 644.   962.   828.  3541. ]
  [ 615.   932.   781.  3420. ]
  [ 614.   865.   816.  3196. ]
  ...
  [ 603.5  805.5  795.  3013.5]
  [ 582.   807.   770.5 3060.5]
  [ 574.5  798.   781.  3071. ]]

 [[ 617.   927.   808.  3608. ]
  [ 590.   904.   800.  3384. ]
  [ 600.5  870.5  823.  3148. ]
  ...
  [ 625.   819.   834.5 2952.5]
  [ 598.   817.   773.  2983.5]
  [ 596.5  816.   782.  2982. ]]

 [[ 636.   920.   824.  3344. ]
  [ 622.   912.   808.  3340. ]
  [ 626.   878.5  823.  3180. ]
  ...
  [ 591.5  785.   790.  2993. ]
  [ 571.   793.   780.  3107. ]
  [ 598.   850.   854.  2998. ]]

 ...

 [[ 788.  1060.   904.  3174. ]
  [ 779.  1061.   904.  3190. ]
  [ 764.  1126.   932.  3516. ]
  ...
  [ 580.   754.   669.  3180. ]
  [ 594.   748.   672.  3233. ]
  [ 546.   754.   722.  3280. ]]

 [[ 807.  1100.   948.  3096. ]
  [ 799.5 1106.   949.  3285. ]
  [ 757.  1050.   881.5 3664. ]
  ...
  [ 682.   825.5  639.  2925. ]
  [ 616.5  778.   609.  3011.5]
  [ 563.5  739.   580.5 3170

In [47]:
rescaled = unstruct.astype(np.float64) / 10000
print(rescaled)


[[[0.0644  0.0962  0.0828  0.3541 ]
  [0.0615  0.0932  0.0781  0.342  ]
  [0.0614  0.0865  0.0816  0.3196 ]
  ...
  [0.06035 0.08055 0.0795  0.30135]
  [0.0582  0.0807  0.07705 0.30605]
  [0.05745 0.0798  0.0781  0.3071 ]]

 [[0.0617  0.0927  0.0808  0.3608 ]
  [0.059   0.0904  0.08    0.3384 ]
  [0.06005 0.08705 0.0823  0.3148 ]
  ...
  [0.0625  0.0819  0.08345 0.29525]
  [0.0598  0.0817  0.0773  0.29835]
  [0.05965 0.0816  0.0782  0.2982 ]]

 [[0.0636  0.092   0.0824  0.3344 ]
  [0.0622  0.0912  0.0808  0.334  ]
  [0.0626  0.08785 0.0823  0.318  ]
  ...
  [0.05915 0.0785  0.079   0.2993 ]
  [0.0571  0.0793  0.078   0.3107 ]
  [0.0598  0.085   0.0854  0.2998 ]]

 ...

 [[0.0788  0.106   0.0904  0.3174 ]
  [0.0779  0.1061  0.0904  0.319  ]
  [0.0764  0.1126  0.0932  0.3516 ]
  ...
  [0.058   0.0754  0.0669  0.318  ]
  [0.0594  0.0748  0.0672  0.3233 ]
  [0.0546  0.0754  0.0722  0.328  ]]

 [[0.0807  0.11    0.0948  0.3096 ]
  [0.07995 0.1106  0.0949  0.3285 ]
  [0.0757  0.105   0.08815

In [48]:
transposed = np.transpose(rescaled, (1, 2, 0))
print(transposed)

[[[0.0644  0.0617  0.0636  ... 0.0788  0.0807  0.086  ]
  [0.0962  0.0927  0.092   ... 0.106   0.11    0.1152 ]
  [0.0828  0.0808  0.0824  ... 0.0904  0.0948  0.0973 ]
  [0.3541  0.3608  0.3344  ... 0.3174  0.3096  0.3366 ]]

 [[0.0615  0.059   0.0622  ... 0.0779  0.07995 0.0818 ]
  [0.0932  0.0904  0.0912  ... 0.1061  0.1106  0.1164 ]
  [0.0781  0.08    0.0808  ... 0.0904  0.0949  0.0954 ]
  [0.342   0.3384  0.334   ... 0.319   0.3285  0.3552 ]]

 [[0.0614  0.06005 0.0626  ... 0.0764  0.0757  0.081  ]
  [0.0865  0.08705 0.08785 ... 0.1126  0.105   0.1058 ]
  [0.0816  0.0823  0.0823  ... 0.0932  0.08815 0.0913 ]
  [0.3196  0.3148  0.318   ... 0.3516  0.3664  0.342  ]]

 ...

 [[0.06035 0.0625  0.05915 ... 0.058   0.0682  0.06985]
  [0.08055 0.0819  0.0785  ... 0.0754  0.08255 0.08345]
  [0.0795  0.08345 0.079   ... 0.0669  0.0639  0.06585]
  [0.30135 0.29525 0.2993  ... 0.318   0.2925  0.3043 ]]

 [[0.0582  0.0598  0.0571  ... 0.0594  0.06165 0.0654 ]
  [0.0807  0.0817  0.0793  ... 0.0

In [49]:
# and then not sure if we need this step, converting to a tf.tensor
# image_raw = tf.io.serialize_tensor(
#     transposed.astype(np.float32)
# )
# print(image_raw)

In [50]:
# how to get numpy array of 4 bands into correct shape for model prediction
reshaped = np.reshape(transposed, (1, 32, 32, 4))
print(reshaped)

[[[[0.0644  0.0617  0.0636  0.0658 ]
   [0.0651  0.0646  0.0643  0.06795]
   [0.0716  0.07045 0.0649  0.0644 ]
   ...
   [0.3304  0.32775 0.3296  0.334  ]
   [0.3416  0.3672  0.384   0.3524 ]
   [0.326   0.3174  0.3096  0.3366 ]]

  [[0.0615  0.059   0.0622  0.0643 ]
   [0.06375 0.0645  0.0642  0.0644 ]
   [0.06665 0.0704  0.0644  0.0637 ]
   ...
   [0.322   0.33095 0.3356  0.3268 ]
   [0.3417  0.3648  0.3728  0.3463 ]
   [0.3348  0.319   0.3285  0.3552 ]]

  [[0.0614  0.06005 0.0626  0.06285]
   [0.064   0.0626  0.0619  0.0626 ]
   [0.0655  0.0696  0.06555 0.0647 ]
   ...
   [0.321   0.3272  0.3308  0.3284 ]
   [0.3413  0.3564  0.3516  0.3428 ]
   [0.3376  0.3516  0.3664  0.342  ]]

  ...

  [[0.06035 0.0625  0.05915 0.05825]
   [0.0599  0.0612  0.0619  0.0632 ]
   [0.0617  0.06035 0.0589  0.0582 ]
   ...
   [0.2951  0.3009  0.31485 0.3148 ]
   [0.3068  0.3204  0.3357  0.3202 ]
   [0.3277  0.318   0.2925  0.3043 ]]

  [[0.0582  0.0598  0.0571  0.0574 ]
   [0.0606  0.0612  0.0621  0.06

In [51]:
# 30-epoch model with 86% binary accuracy
model_name = "resnet"
optimizer = "adam"
loss_function = "binary_crossentropy"
checkpoint = "C:\\fao-models\\saved_models\\resnet-epochs30-batch64-lr001\\best_model.h5"

# load several model versions into memory..
model = get_model(model_name, optimizer=optimizer, loss_fn=loss_function)
model.load_weights(checkpoint)
# when mode.trainable was set to False, was getting error at model.load_weights(checkpoint): ValueError: axes don't match array
# https://stackoverflow.com/questions/51944836/keras-load-model-valueerror-axes-dont-match-array
model.trainable = True 

# print(model.summary())
prediction = model(reshaped)
print(prediction)

Model found: resnet
tf.Tensor([[0.42960128]], shape=(1, 1), dtype=float32)


In [52]:
# 15 epoch model with 98% binary accuracy 
model_name = "resnet"
optimizer = "adam"
loss_function = "binary_crossentropy"
checkpoint = "C:\\fao-models\\saved_models\\resnet-epochs5-batch64-lr001-seed5-lrdecay5\\best_model.h5"
# load several model versions into memory..
model = get_model(model_name, optimizer=optimizer, loss_fn=loss_function)
model.load_weights(checkpoint)
# when mode.trainable was set to False, was getting error at model.load_weights(checkpoint): ValueError: axes don't match array
# https://stackoverflow.com/questions/51944836/keras-load-model-valueerror-axes-dont-match-array
model.trainable = True 

# print(model.summary())
prediction = model(reshaped)
print(prediction)


Model found: resnet
tf.Tensor([[0.47269127]], shape=(1, 1), dtype=float32)
